# Recommendation System Notebook
- User based recommendation
- User based prediction
- Item based recommendation
- Item based prediction
- Evaluation

In [1]:
# import libraties
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# DATASET LINK : https://grouplens.org/datasets/movielens/
ratings = pd.read_csv('ratings.csv', encoding='latin-1')

In [2]:
ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


## Dividing the dataset into train and test

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.30, random_state=31)

In [4]:
print(train.shape)
print(test.shape)

(185153, 4)
(79352, 4)


In [5]:
# pivot ratings into movie features
df_movie_features = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [6]:
df_movie_features.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.5,0.0,0.0,0.0,3.0,4.5,3.5,3.0
910,0.0,4.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0


### Copy train and test dataset
These dataset will be used for prediction and evaluation. 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction. 
- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train

In [7]:
dummy_train = train.copy()
dummy_test = test.copy()

In [8]:
dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x>=1 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x>=1 else 0)

In [9]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train = dummy_train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(1)

# The movies not rated by user is marked as 0 for evaluation. 
dummy_test = dummy_test.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [10]:
dummy_train.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
320,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
359,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
370,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
910,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [11]:
dummy_test.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
370,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
910,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# User Similarity Matrix

## Using Cosine Similarity

In [12]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(df_movie_features, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[1.         0.27437326 0.20805722 ... 0.21701259 0.         0.15886221]
 [0.27437326 1.         0.16309164 ... 0.15044338 0.         0.13787661]
 [0.20805722 0.16309164 1.         ... 0.26569469 0.02849447 0.23968733]
 ...
 [0.21701259 0.15044338 0.26569469 ... 1.         0.044708   0.14407477]
 [0.         0.         0.02849447 ... 0.044708   1.         0.0174666 ]
 [0.15886221 0.13787661 0.23968733 ... 0.14407477 0.0174666  1.        ]]


In [13]:
user_correlation.shape

(862, 862)

## Using adjusted Cosine 

### Here, not removing the NaN values and calculating the mean only for the movies rated by the user

In [14]:
movie_features = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
)

In [15]:
movie_features.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,4.5,NaN,NaN,NaN,3.0,4.5,3.5,3.0
910,NaN,4.0,NaN,NaN,NaN,3.5,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN


### Normalising the rating of the movie for each user aroung 0 mean

In [16]:
mean = np.nanmean(movie_features, axis=1)
df_subtracted = (movie_features.T-mean).T

In [17]:
df_subtracted.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.847368,NaN,NaN,NaN,NaN,NaN,-1.347368,NaN,-0.847368,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315700,0.315700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.790441,0.709559,NaN,NaN,NaN,-0.790441,0.709559,-0.290441,-0.790441
910,NaN,0.087607,NaN,NaN,NaN,-0.412393,NaN,NaN,NaN,0.087607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587607,NaN,NaN


### Finding cosine similarity

In [18]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[ 1.          0.113453    0.11494838 ...  0.07731154  0.
  -0.04190499]
 [ 0.113453    1.          0.07188267 ...  0.07301337  0.
   0.02765892]
 [ 0.11494838  0.07188267  1.         ...  0.06492798  0.08561282
  -0.01730643]
 ...
 [ 0.07731154  0.07301337  0.06492798 ...  1.          0.05669439
   0.03825774]
 [ 0.          0.          0.08561282 ...  0.05669439  1.
  -0.00102823]
 [-0.04190499  0.02765892 -0.01730643 ...  0.03825774 -0.00102823
   1.        ]]


## Prediction

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [19]:
user_correlation[user_correlation<0]=0
user_correlation

array([[1.        , 0.113453  , 0.11494838, ..., 0.07731154, 0.        ,
        0.        ],
       [0.113453  , 1.        , 0.07188267, ..., 0.07301337, 0.        ,
        0.02765892],
       [0.11494838, 0.07188267, 1.        , ..., 0.06492798, 0.08561282,
        0.        ],
       ...,
       [0.07731154, 0.07301337, 0.06492798, ..., 1.        , 0.05669439,
        0.03825774],
       [0.        , 0.        , 0.08561282, ..., 0.05669439, 1.        ,
        0.        ],
       [0.        , 0.02765892, 0.        , ..., 0.03825774, 0.        ,
        1.        ]])

Rating predicted by the user (for movies rated as well as not rated) is the weighted sum of correlation with the movie rating (as present in the rating dataset). 

In [20]:
user_predicted_ratings = np.dot(user_correlation, movie_features.fillna(0))
user_predicted_ratings

array([[50.25431628, 23.21431421,  6.92774725, ..., 10.29310194,
         5.18963663, 10.30393691],
       [63.1668671 , 28.66715852,  7.70238561, ..., 10.94090127,
         7.23656686, 13.43793905],
       [65.8132159 , 31.28951501,  8.87591053, ..., 13.18816596,
         7.40423075, 13.93635509],
       ...,
       [90.84386199, 45.29729897, 14.90281743, ..., 14.53319001,
        10.03775996, 17.92253468],
       [40.08352   , 17.48819858,  5.61950399, ...,  5.25427855,
         4.14222761,  5.40077794],
       [42.21672064, 21.86231082,  5.81298622, ...,  5.1213186 ,
         4.37796843,  6.21908569]])

In [21]:
user_predicted_ratings.shape

(862, 2500)

Since we are interested only in the movies not rated by the user, we will ignore the movies rated by the user by making it zero. 

In [22]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,23.214314,6.927747,0.549306,6.464053,23.535059,0.000000,2.235682,0.000000,10.623249,...,7.328997,7.604004,7.196259,8.239460,9.917107,7.476424,7.683810,10.293102,5.189637,10.303937
320,63.166867,28.667159,7.702386,1.141707,7.491681,30.553140,9.110414,2.127390,33.996904,11.592103,...,9.891204,9.906554,11.141293,12.707200,14.315709,11.806246,8.871086,10.940901,7.236567,13.437939
359,65.813216,31.289515,8.875911,0.875243,9.167411,33.674505,10.068789,2.861450,0.000000,0.000000,...,11.181333,10.368695,11.039097,12.413749,13.442644,10.815282,11.196572,13.188166,7.404231,13.936355
370,54.516870,27.369566,5.724738,0.685778,6.437963,26.894367,7.839079,2.315056,29.568027,10.915500,...,10.899825,0.000000,0.000000,12.529633,13.884563,12.486731,0.000000,0.000000,0.000000,0.000000
910,31.147689,0.000000,4.950286,0.461965,5.406109,0.000000,6.736494,1.663783,15.614400,0.000000,...,5.734458,4.455202,4.900093,5.007222,5.803241,4.223923,4.302043,0.000000,4.063610,5.529644


### Finding the top 5 recommendation for the user 1 

In [23]:
user_final_rating.iloc[1].sort_values(ascending=False)[0:5]

movieId
296     107.287371
356      93.704186
7153     90.188210
318      87.105961
5952     86.462804
Name: 320, dtype: float64

# Item Based Similarity

Using Correlation

Taking the transpose of the rating matrix to normalize the rating around the mean for different movie ID. In the user based similarity, we had taken mean for each user intead of each movie. 

In [24]:
movie_features = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).T

movie_features.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,
1,2.5,NaN,NaN,NaN,NaN,NaN,4.0,3.0,4.5,NaN,...,3.0,NaN,NaN,NaN,3.5,NaN,NaN,3.0,NaN,3.0
2,NaN,NaN,NaN,NaN,4.0,NaN,3.0,3.0,NaN,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
